In [2]:
using DataFrames
using JuMP, Gurobi

In [3]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

In [4]:
function GetC()

    W_ = 19

    C = zeros((D*W_, D*W_))

    for t in 1:(W_*D)
        for t_ in 1:(W_*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [10]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0, Monday=0.0)
    for i=1:N
        for j=1:N
            for d=1:D
                if x[i,j,week,d] == 1.0
                        df[i, 1+d] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [12]:
function Optimize4WeeksAndFreeze(X_old, Y_old, W)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    W = W # weeks
    D = 7 # days

    print(1)

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = 1:W, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = 1:W, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    print(2)

    # OBJECTIVE FUNCTION
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in 1:W-1, d in 1:D, d_ in 1:D))

    print(3)

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = 1:W-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)

    print(4)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = 1:W, d = 1:D], x[i, i, w, d] == 0)

    print(5)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    print(6)

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = 1:W], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    print(7)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in 1:W, d in 1:D) == W)

    print(8)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in 1:W, d in 1:D) <= 1)

    print(9)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = 1:W))

    print(10)

    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in 1:W, d in 1:D) <= 1 - Y_old[i, j])

    print(11)

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = 1:W, d = 1:3], x[i, j, w, d] == 0)

    print(12)

    # OPTIMIZE
    optimize!(model)

    print(13)

    # update old X, Y before returning them
    for i in 1:N
        for j in 1:N
            for w in 1:W
                for d in 1:D
                    X_old[i, j, w, d] = value.(x[i, j, w, d])
                    Y_old[i,j] = value.(y[i,j])
                end
            end
        end
    end

    print(14)

    return X_old, Y_old


end


Optimize4WeeksAndFreeze (generic function with 1 method)

In [13]:
W = 4

# first time, X_old and Y_old initialized to empty
X_old = zeros(20, 20, W, 7) # teams are never playing against each other
Y_old = zeros(20, 20)   # teams never played against each other before
;

In [14]:
# optimize for 4 weeks
X, y = Optimize4WeeksAndFreeze(X_old, Y_old, W);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 600
12

34

56789101112

Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3547060 rows, 1579600 columns and 8349840 nonzeros
Model fingerprint: 0x20f62528
Variable types: 1568000 continuous, 11600 integer (11600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1727330 rows and 970080 columns (presolve time = 5s) ...
Presolve removed 1727330 rows and 970080 columns (presolve time = 10s) ...
Presolve removed 1727330 rows and 970080 columns (presolve time = 15s) ...
Presolve removed 1727330 rows and 970080 columns (presolve time = 20s) ...
Presolve removed 1727330 rows and 970080 columns (presolve time = 25s) ...
Presolve removed 1727330 rows and 970080 columns (presolve time = 92s) ...
Presolve removed 2940290 rows and 970080 columns (

In [19]:
for w in 1:W
    display(calendar_week(X, w))
end

,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,9.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,13.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,5,0.0,0.0,0.0,14.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,16.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,19.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,18.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,1.0,0.0,0.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,8.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,18.0
3,3,0.0,0.0,0.0,0.0,12.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,11.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,10.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,13.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,15.0
8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,14.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,6.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,14.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,9.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,3.0,0.0
7,7,0.0,0.0,0.0,20.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,15.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,5.0,0.0


,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,18.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,4,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,19.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,17.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,13.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,12.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,16.0


In [38]:
# for i = 1:20
#     for j = 1:20
#         for w = 1:W
#             for d = 1:7
#                 if X[i, j, w, d] == 1.0
#                     println("Team ", i, " plays team ", j, " on day ", d, " of week ", w)
#                 end
#             end
#         end
#     end
# end

In [40]:
# y